<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/minjoo/sts_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('/content/drive/MyDrive/wanted')

In [ ]:
!ls

dataset  sts_assignment.ipynb


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 3.8 MB 12.6 MB/s 
     |████████████████████████████████| 1.2 MB 37.6 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 31.7 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 6.5 MB 27.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=a72f958eb5efa10473c2ff87c402b39cfd7bb467621ec96886e609116d14d3ea
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
"""
This examples trains KoBERT for the STS benchmark from scratch.
It generates sentence embeddings that can be compared using cosine-similarity to measure the similarity.
Usage:
python training_sts.py --model_name_or_path klue/bert-base
"""
import argparse
import logging
import math
import os
import random
from datetime import datetime

import numpy as np
import torch
from sentence_transformers import SentenceTransformer, LoggingHandler, models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader

import json
from sentence_transformers.readers import InputExample
# from data_util import load_kor_sts_samples

In [ ]:
# Parse command line arguments
parser = argparse.ArgumentParser()
parser.add_argument("--model_name_or_path", type=str, default = "klue/roberta-base")
parser.add_argument("--max_seq_length", type=int, default=128)
parser.add_argument("--batch_size", type=int, default=8)
parser.add_argument("--num_epochs", type=int, default=5)
parser.add_argument("--output_dir", type=str, default="output")
parser.add_argument("--output_prefix", type=str, default="kor_sts_")
parser.add_argument("--seed", type=int, default=777)
args = parser.parse_args("")

In [ ]:
# Fix random seed
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# Configure logger
logging.basicConfig(
    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO, handlers=[LoggingHandler()]
)

# Read the dataset
model_save_path = os.path.join(
    args.output_dir,
    args.output_prefix + args.model_name_or_path.replace("/", "-") + '-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [ ]:
# Define SentenceTransformer model
word_embedding_model = models.Transformer(args.model_name_or_path, max_seq_length=args.max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device = 'cuda')

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
def load_kor_sts_samples(filename):
    samples = []
    with open(filename,'r') as f:
        json_data = json.load(f)
    for row in json_data:
        score = float(row['labels']['label']) / 5.0  # Normalize score to range 0 ... 1
        samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
    return samples

In [ ]:
# Read the dataset
logging.info("Read KorSTS train/dev dataset")

train_file = './dataset/klue-sts-v1.1_train.json'
train_samples = load_kor_sts_samples(train_file)
random.shuffle(train_samples)

n_train = int(0.9 * len(train_samples))
valid_samples = train_samples[n_train:]
train_samples = train_samples[:n_train]

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=args.batch_size)
valid_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_samples, batch_size=args.batch_size,
                                                                 name='sts-valid')

2022-03-14 09:37:04 - Read KorSTS train/dev dataset


In [ ]:
train_loss = losses.CosineSimilarityLoss(model=model)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * args.num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2022-03-14 09:37:07 - Warmup-steps: 657


In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=valid_evaluator,
          epochs=args.num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1313 [00:00<?, ?it/s]

2022-03-14 09:43:02 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 0 after 1000 steps:
2022-03-14 09:43:13 - Cosine-Similarity :	Pearson: 0.9491	Spearman: 0.9080
2022-03-14 09:43:13 - Manhattan-Distance:	Pearson: 0.9455	Spearman: 0.9073
2022-03-14 09:43:13 - Euclidean-Distance:	Pearson: 0.9458	Spearman: 0.9075
2022-03-14 09:43:13 - Dot-Product-Similarity:	Pearson: 0.9384	Spearman: 0.8900
2022-03-14 09:43:13 - Save model to output/kor_sts_klue-roberta-base-2022-03-14_09-32-38
2022-03-14 09:44:58 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 0:
2022-03-14 09:45:09 - Cosine-Similarity :	Pearson: 0.9532	Spearman: 0.9095
2022-03-14 09:45:09 - Manhattan-Distance:	Pearson: 0.9490	Spearman: 0.9120
2022-03-14 09:45:09 - Euclidean-Distance:	Pearson: 0.9488	Spearman: 0.9116
2022-03-14 09:45:09 - Dot-Product-Similarity:	Pearson: 0.9414	Spearman: 0.8918
2022-03-14 09:45:09 - Save model to output/kor_sts_klue-roberta-base-20

Iteration:   0%|          | 0/1313 [00:00<?, ?it/s]

...

2022-03-14 09:58:17 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 2 after 1000 steps:
2022-03-14 09:58:28 - Cosine-Similarity :	Pearson: 0.9589	Spearman: 0.9202
2022-03-14 09:58:28 - Manhattan-Distance:	Pearson: 0.9538	Spearman: 0.9199
2022-03-14 09:58:28 - Euclidean-Distance:	Pearson: 0.9541	Spearman: 0.9203
2022-03-14 09:58:28 - Dot-Product-Similarity:	Pearson: 0.9463	Spearman: 0.8993
2022-03-14 09:58:28 - Save model to output/kor_sts_klue-roberta-base-2022-03-14_09-32-38
2022-03-14 10:00:13 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 2:
2022-03-14 10:00:24 - Cosine-Similarity :	Pearson: 0.9591	Spearman: 0.9198
2022-03-14 10:00:24 - Manhattan-Distance:	Pearson: 0.9526	Spearman: 0.9173
2022-03-14 10:00:24 - Euclidean-Distance:	Pearson: 0.9527	Spearman: 0.9180
2022-03-14 10:00:24 - Dot-Product-Similarity:	Pearson: 0.9485	Spearman: 0.9040


Iteration:   0%|          | 0/1313 [00:00<?, ?it/s]

2022-03-14 10:05:52 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 3 after 1000 steps:
2022-03-14 10:06:03 - Cosine-Similarity :	Pearson: 0.9589	Spearman: 0.9186
2022-03-14 10:06:03 - Manhattan-Distance:	Pearson: 0.9523	Spearman: 0.9170
2022-03-14 10:06:03 - Euclidean-Distance:	Pearson: 0.9527	Spearman: 0.9178
2022-03-14 10:06:03 - Dot-Product-Similarity:	Pearson: 0.9467	Spearman: 0.9007
2022-03-14 10:07:47 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 3:
2022-03-14 10:07:58 - Cosine-Similarity :	Pearson: 0.9595	Spearman: 0.9212
2022-03-14 10:07:58 - Manhattan-Distance:	Pearson: 0.9521	Spearman: 0.9193
2022-03-14 10:07:58 - Euclidean-Distance:	Pearson: 0.9524	Spearman: 0.9197
2022-03-14 10:07:58 - Dot-Product-Similarity:	Pearson: 0.9461	Spearman: 0.9023
2022-03-14 10:07:58 - Save model to output/kor_sts_klue-roberta-base-2022-03-14_09-32-38


Iteration:   0%|          | 0/1313 [00:00<?, ?it/s]

2022-03-14 10:13:29 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 4 after 1000 steps:
2022-03-14 10:13:40 - Cosine-Similarity :	Pearson: 0.9605	Spearman: 0.9219
2022-03-14 10:13:40 - Manhattan-Distance:	Pearson: 0.9531	Spearman: 0.9199
2022-03-14 10:13:40 - Euclidean-Distance:	Pearson: 0.9535	Spearman: 0.9204
2022-03-14 10:13:40 - Dot-Product-Similarity:	Pearson: 0.9480	Spearman: 0.9029
2022-03-14 10:13:40 - Save model to output/kor_sts_klue-roberta-base-2022-03-14_09-32-38
2022-03-14 10:15:26 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 4:
2022-03-14 10:15:37 - Cosine-Similarity :	Pearson: 0.9608	Spearman: 0.9221
2022-03-14 10:15:37 - Manhattan-Distance:	Pearson: 0.9530	Spearman: 0.9199
2022-03-14 10:15:37 - Euclidean-Distance:	Pearson: 0.9535	Spearman: 0.9205
2022-03-14 10:15:37 - Dot-Product-Similarity:	Pearson: 0.9485	Spearman: 0.9035
2022-03-14 10:15:37 - Save model to output/kor_sts_klue-roberta-base-20

In [ ]:
# Load the stored model and evaluate its performance on STS benchmark dataset
path = '/content/drive/MyDrive/wanted/output/kor_sts_klue-roberta-base-2022-03-14_09-32-38'
model = SentenceTransformer(path)
logging.info("Read KorSTS benchmark test dataset")
test_file = './dataset/klue-sts-v1.1_dev.json'
test_samples = load_kor_sts_samples(test_file)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=path)

2022-03-14 11:26:53 - Load pretrained SentenceTransformer: /content/drive/MyDrive/wanted/output/kor_sts_klue-roberta-base-2022-03-14_09-32-38
2022-03-14 11:26:55 - Use pytorch device: cuda
2022-03-14 11:26:55 - Read KorSTS benchmark test dataset
2022-03-14 11:26:55 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-03-14 11:26:59 - Cosine-Similarity :	Pearson: 0.8887	Spearman: 0.8911
2022-03-14 11:26:59 - Manhattan-Distance:	Pearson: 0.8884	Spearman: 0.8863
2022-03-14 11:26:59 - Euclidean-Distance:	Pearson: 0.8889	Spearman: 0.8864
2022-03-14 11:26:59 - Dot-Product-Similarity:	Pearson: 0.8672	Spearman: 0.8652


0.8911038886297658